In [1]:
from elasticsearch import Elasticsearch
import requests
from  elasticsearch.helpers import scan as scan

es = Elasticsearch([{"host": "localhost", "port": 9200}])

In [2]:
import pandas as pd
from pandas.io.json import json_normalize
#df=json_normalize(docs)

In [26]:
body={
    "_source": [
        "context.user_id",
        "context.course_id",
        "time",
        "event.submission",
        "event.attempts",
        "event.problem_id" 
    ],
    "query": {
        "bool":{
            "must":[                
                {"match_phrase":{
                    "event_type": "problem_check"
                }},
                {"match":{
                    "event_source": "server"
                }}           
            ],
            "must_not":[
                {"match_phrase":{
                    "event_type": "problem_check_fail"
                }}
            ]            
        }
    }
}

def ret():
    for hit in scan(es,
         index="online_courses", 
         query=body,
         sort={"time":"desc"}
        ):        
        if hit.get("_source").get("event") :
            if hit.get("_source").get("event").get("submission")!={}:
                submission=str(hit.get("_source").get("event").get("submission")).split('correct\': ')[1:]
                results=[]
                for res in submission:
                    if res[0:5]=='False':
                        results.append(0)
                    elif res[0:4]=='True':
                        results.append(1)
                yield {                    
                    "user_id":hit.get("_source").get("context").get("user_id"),
                    "course_id":hit.get("_source").get("context").get("course_id"),
                    "time":hit.get("_source").get("time"),
                    "results":results,
                    "attempt_num":hit.get("_source").get("event").get("attempts"),
                    "problem_id":hit.get("_source").get("event").get("problem_id"),
        }
            
        
        


In [27]:
df=pd.DataFrame(ret()).sort_values(by=['time'])

In [28]:
df[:3]

,attempt_num,course_id,problem_id,results,time,user_id
0,1,course-v1:spbu+SOCLAB+spring_2018,block-v1:spbu+SOCLAB+spring_2018+type@problem+...,"[0, 0, 1, 0, 0]",2018-02-15T09:05:34.388942+00:00,243054
1,1,course-v1:spbu+SOCLAB+spring_2018,block-v1:spbu+SOCLAB+spring_2018+type@problem+...,"[0, 0, 0, 0, 0]",2018-02-15T12:41:49.105627+00:00,492510
3,2,course-v1:spbu+SOCLAB+spring_2018,block-v1:spbu+SOCLAB+spring_2018+type@problem+...,"[0, 0, 0, 0, 0]",2018-02-15T12:45:45.648085+00:00,492510


In [29]:
df.to_csv(path_or_buf='tests_results.csv')